# Import Package

In [ ]:
!pip install stockstats

In [ ]:

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from stockstats import StockDataFrame

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

from math import floor

import pandas_datareader.data as web
import pickle
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model = "CNN_LSTM"

volatility_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/Volatility_result/"
backtest_summary_folder = volatility_folder + "backtest_summary/"
record_folder = volatility_folder + "trade_record/"
backtest_folder= "/content/drive/MyDrive/fyp/Final report/4. Backtesting/"
model_folder = backtest_folder+model+"/"

Mounted at /content/drive


# Backtesting Setting

In [ ]:
start_date = []
num_period = 10
period_year = 1

start = 2022-(num_period+period_year-1)
for i in range(num_period):
    start_date.append((start+i)*10000+101)


date_range_list={}
for i in start_date:
    date_range_list[i] = (i-101)+(period_year-1)*10000+1231

print(date_range_list)

stock_list = {"BlockChain":["COIN","NVDA","FB"],
              "Airline":["BA","GD","LMT",],
              "Traveling":["UBER","ABNB","MAR","BKNG"],
              "Semiconductors":["INTC","NVDA","QCOM","MU","AMD"],
              "Cloud Computing":["IBM", "AMZN","GOOG","CRM"],
              "Social Media":["TWTR","SNAP","PINS","FB"],
              "Entertainment":["DIS","NFLX","FB"],
              "Retail":["WMT","COST","TGT","BBY","HD"],
              "Franchise":["MCD", "YUM", "SBUX", "DPZ"],
              "Real Estate":["HST", "EQR", "AVB", "PLD", "SPG"],
              "Telecommunication":["T","TMUS","VZ","CMCSA","CHTR"],
              "Energy & Resources":["DOW", "DD"],
              "Luxury goods":["RACE","EL","PVH"]
                            }

start_date = "2000-01-01"
end_date = "2022-12-31"


do_nth_name = "Buy&Hold"
model_name = "WithModel"

backtest_col = ["Sector","Stock",
                "Range_Start","Range_End",
                do_nth_name+"_Profit",model_name+"_Profit",
                do_nth_name+"_CAGR(in %)",model_name+"_CAGR(in %)",
                "CAGR_Performance(in %)",
                do_nth_name+"_MDD(in %)",model_name+"_MDD(in %)",
                "MDD_Performance(in %)"]

{20120101: 20121231, 20130101: 20131231, 20140101: 20141231, 20150101: 20151231, 20160101: 20161231, 20170101: 20171231, 20180101: 20181231, 20190101: 20191231, 20200101: 20201231, 20210101: 20211231}


# Backtesting Module


In [ ]:
################### Input ##########################
# For hist_price_data: index=["date"], columns = ["Open"]
# For pred_action: index=["date"], columns = ["Action"] (Buy/Sell)
################### Output #########################
# 1. trading record
# 2. total profit
class backtest:
    hpd = ""
    pred_action=pd.DataFrame()
    trade_record=pd.DataFrame(index=[],
                              columns=["Action","Price","Position","Cash","Pos_Bal","Cash_Bal","Cum_Profit","Total_Bal"],
                             )
    capital = 0
    cash_balance = 0
    profit = 0
    handle_fee = 0
    position = 0
    last_price = 0
    do_nth_profit = 0
    num_year = 0
    _tested = False
    
    _stock_trough = 0
    _stock_peak = 0
    _stock_all_time_low = 0
    _stock_all_time_high = 0
    
    _portfolio_trough = 0
    _portfolio_peak = 0
    _portfolio_all_time_low = 0
    _portfolio_all_time_high = 0
    
    def __init__(self,hist_price_data,pred_action,capital,handling_fee,num_year=1):
        self.hpd = hist_price_data
        self.pred_action = pred_action
        self.capital = capital
        self.cash_balance = capital
        self._portfolio_trough = capital
        self._portfolio_peak = capital
        self._portfolio_all_time_low = capital
        self._portfolio_all_time_high = capital
        self.handle_fee = handling_fee
        self.num_year = num_year  
        
    def clear_trade_record(self,sec):
        self.trade_record=pd.DataFrame(index=[],
                                       columns=["Action","Price","Position","Cash","Pos_Bal","Cash_Bal","Cum_Profit","Total_Bal"],
                                       )
        print("Clearing trade_record...")
        time.sleep(sec)
        
    def start_test(self): 
        if not self._tested:
            status = "sell"
            self._tested = True
            print("Start Backtesting...")  
            self._stock_all_time_low = self.hpd.iloc[0,0]
            self._stock_all_time_high = self.hpd.iloc[0,0]
            self._stock_trough = self.hpd.iloc[0,0]
            self._stock_peak = self.hpd.iloc[0,0]
            # For loop to iterate the data
            for ind in self.pred_action.index:
                # Update latest price
                self.last_price = self.hpd.loc[ind,"Open"]
                
                # Mark All Time Low,High , Trough and Peak for MDD of stock price (using "Buy&Hold")
                if self.last_price > self._stock_all_time_high:
                    self._stock_all_time_high = self.last_price
                if self.last_price < self._stock_all_time_low:
                    self._stock_all_time_low = self.last_price
                if self.last_price > self._stock_all_time_low and self._stock_trough != self._stock_all_time_low:
                    self._stock_trough = self._stock_all_time_low
                if self.last_price < self._stock_all_time_high and self._stock_peak != self._stock_all_time_high:
                    self._stock_peak = self._stock_all_time_high
                
                # Do the action
                if self.pred_action.loc[ind,"Action"].lower() == "buy" and status == "sell":
                    self._buy(ind,self.last_price)
                    status = "buy"
                elif self.pred_action.loc[ind,"Action"].lower() == "sell" and status == "buy":
                    self._sell(ind,self.last_price)
                    status = "sell"
                else:
                    self._hold(ind,self.last_price) # newly added
                    
                
                curr_portfolio_val = self.get_capital()+self.get_profit()
                
                # Mark All Time Low,High , Trough and Peak for MDD of our portfolio balance (using our model)
                if curr_portfolio_val > self._portfolio_all_time_high:
                    self._portfolio_all_time_high = curr_portfolio_val
                if curr_portfolio_val < self._portfolio_all_time_low:
                    self._portfolio_all_time_low = curr_portfolio_val
                if curr_portfolio_val > self._portfolio_all_time_low and self._portfolio_trough != self._portfolio_all_time_low:
                    self._portfolio_trough = self._portfolio_all_time_low
                if curr_portfolio_val < self._portfolio_all_time_high and self._portfolio_peak != self._portfolio_all_time_high:
                    self._portfolio_peak = self._portfolio_all_time_high
                            
            # =================================================
            self._run_do_nothing() # Calculate do nothing profit
        else:
            print("Backtesting has been completed...")
              
                
        
    def _mark_down_record(self,date,action,price,pos_delta,cash_delta):
        self.trade_record.loc[date] = [action,price,pos_delta,cash_delta,
                                       round(self.position,4),round(self.cash_balance,3),
                                       round(self.get_profit(),3),round(self.get_amount(),3)]
        
    def _buy(self,date,price):
        # Assume use all money to buy all
        buy_pos = floor(self.cash_balance / price)
        for i in range(buy_pos+1):
            act_buy_pos = buy_pos - i
            if act_buy_pos == 0:
                #print("You do not have enough money to buy!")
                return
            total_amt = act_buy_pos*price*(1+self.handle_fee)
            if self.cash_balance > total_amt:
                self.position += act_buy_pos
                self.cash_balance -= total_amt
                self._mark_down_record(date,
                                       "Buy",
                                       price,
                                       act_buy_pos,
                                       -total_amt)
                print("Bought at",date,"with price =", price, "\tPos:", act_buy_pos)
                return
        
    
    def _sell(self,date,price):
        # Assume sell all position
        sell_pos = self.position
        total_amt = sell_pos*price*(1-self.handle_fee)
        if self.position >= 1:
            self.position -= sell_pos
            self.cash_balance += total_amt
            self._mark_down_record(date,
                                   "Sell",
                                   price,
                                   -sell_pos,
                                   total_amt)
            print("Sold at",date,"with price =", price, "\tPos:", sell_pos)
            
            return
        
    def _hold(self,date,price): # newly added
        self._mark_down_record(date,"Hold",price,0,0)

    def _run_do_nothing(self):
        balance = self.capital
        first_day_price = self.hpd.iloc[0,0]
        last_day_price = self.hpd.iloc[-1,0]
        buy_pos = floor(self.capital / first_day_price) # Calculate how many position can buy
        total_amt = buy_pos*first_day_price*(1+self.handle_fee) # Check if okay to buy (including fee)
        while self.capital < total_amt: # If not enough, reduce buy_pos by 1
            buy_pos -= 1
            total_amt = buy_pos*first_day_price*(1+self.handle_fee)
        position = buy_pos # Buy in 
        balance -= total_amt
        self.do_nth_profit = last_day_price*position+balance-self.capital

    def get_performance(self):
        if self._tested:
            model_profit = self.get_profit()
            do_nth_profit = self.get_do_nothing()
            perf = (model_profit-do_nth_profit)/abs(do_nth_profit)
            return perf
        else:
            print("No Backtesting Record.")

    def get_do_nothing(self):
        if self._tested:
            return self.do_nth_profit
        else:
            print("No Backtesting Record.")

    def get_profit(self):
        if self._tested:
            return self.get_cash_balance()+self.get_last_price()*self.get_position()-self.get_capital()
        else:
            print("No Backtesting Record.")

    def get_last_price(self):
        if self._tested:
            return self.last_price
        else:
            print("No Backtesting Record.")
    
    def get_position(self):
        if self._tested:
            return self.position
        else:
            print("No Backtesting Record.")
    
    def get_do_nothing_CAGR(self):
        if self._tested:
            start_bal = self.get_capital()
            end_bal = self.get_do_nothing()+self.get_capital()
            num_year = self.get_num_year()
            return pow(end_bal/start_bal,1/num_year)-1
        else:
            print("No Backtesting Record.")
            
    def get_model_CAGR(self):
        if self._tested:
            start_bal = self.get_capital()
            end_bal = self.get_profit()+self.get_capital()
            num_year = self.get_num_year()
            return pow(end_bal/start_bal,1/num_year)-1
        else:
            print("No Backtesting Record.")
            
    def get_diff_in_CAGR(self):
        if self._tested:
            return self.get_model_CAGR() - self.get_do_nothing_CAGR()
        else:
            print("No Backtesting Record.")
    
    def get_do_nothing_MDD(self):
        if self._tested:
            return (self._stock_trough-self._stock_peak)/self._stock_peak
        else:
            print("No Backtesting Record.")
            
    def get_model_MDD(self):
        if self._tested:
            return (self._portfolio_trough-self._portfolio_peak)/self._portfolio_peak
        else:
            print("No Backtesting Record.")
            
    def get_diff_in_MDD(self):
        if self._tested:
            return self.get_model_MDD() - self.get_do_nothing_MDD()
        else:
            print("No Backtesting Record.")
    
    def get_num_year(self):
        return self.num_year
    
    def get_cash_balance(self):
        return self.cash_balance
    
    def get_capital(self):
        return self.capital
    
    def get_amount(self):
        return self.get_capital()+self.get_profit()

    def print_do_nothing(self):
        if self._tested:
            print("If buy at", self.hpd.index[0],"with price =",self.hpd.iloc[0,0])
            print("and do nothing")
            print("Current Profit:",self.get_do_nothing())
        else:
            print("No Backtesting Record.")
            
    def print_performance(self):
        if self._tested:
            print("Performance:", str(round(self.get_performance()*100,2))+"%")
        else:
            print("No Backtesting Record.")
            
    def print_CAGR_performance(self):
        if self._tested:
            print("CAGR Performance:", str(round(self.get_diff_in_CAGR()*100,2))+"%")
        else:
            print("No Backtesting Record.")
    
    def print_trade_record(self):
        if self._tested:
            print(self.trade_record)
        else:
            print("No Backtesting Record.")
            print(self.trade_record)
    
    def print_profit(self):
        if self._tested:
            print("Current Profit with model:",self.get_profit())
        else:
            print("No Backtesting Record.")
    
    def export_trade_record(self,stock,add_msg = ""):
        if self._tested:
            # Save the trade record to the path
            if not add_msg == "":
                add_msg = "_" + add_msg
            self.trade_record = self.trade_record.sort_index()
            self.trade_record.to_csv(record_folder+stock+add_msg+".csv")
            print("Trade record exported.")
        else:
            print("No Backtesting Record.")

# Processing Function

In [ ]:
# Split the train and test data
def custom_split(data,start,end):
    train = (data.index >= start) & (data.index <= end)
    train_X = data[train]
    
    return train_X

In [ ]:
# Assume we use 5 days price data to predict opening price of the 6th day
num_day_to_predict = 5


In [ ]:
def produce_result_target_price(X,num_day,result_col_name = "result_price"):
    y = pd.DataFrame(np.nan, index=X.index, columns=[result_col_name])
    for i in range(len(X)-num_day):
        y.iloc[i+num_day_to_predict,0] = X.iloc[i+num_day,0]
    return y

In [ ]:
def transform_data_to_tensor(X,y,num_day):
    # Initiate tensor for X
    x_first = X.iloc[0:num_day,:]
    x_mean = x_first.mean(axis=0) # Get the mean of the 10-day frame
    x_std = x_first.std(axis=0) # Get the std of the 10-day frame
    x_first = x_first.sub(x_mean, axis=1).div(x_std, axis=1) # Normalize the 10-day frame here
    
    # Initiate tensor for y
    x_open = X.iloc[0:num_day,0]
    y_val = y.iloc[num_day,:] # Get the corresponding y
    y_val = y_val.sub(x_open.mean(axis=0)).div(x_open.std(axis=0)) # Normalize the y
    
    x_tf_data = [tf.convert_to_tensor(np.array(x_first),dtype = tf.float32)]
    y_tf_data = [tf.convert_to_tensor(np.array(y_val),dtype = tf.float32)]
    
    for i in range(1,len(X)-num_day):   
        x_window = X.iloc[i:i+num_day,:] # Set the window as a 10-day frame 
        x_mean = x_window.mean(axis=0) # Get the mean of the 10-day frame
        x_std = x_window.std(axis=0) # Get the std of the 10-day frame
        x_window = x_window.sub(x_mean, axis=1).div(x_std, axis=1) # Normalize the 10-day frame here
        
        x_open = X.iloc[i:i+num_day,0] # Get the opening price of the 10-day frame
        y_val = y.iloc[i+num_day,:] # Get the corresponding y
        y_val = y_val.sub(x_open.mean(axis=0)).div(x_open.std(axis=0)) # Normalize the y
        
        x_next_tf = tf.convert_to_tensor(np.array(x_window),dtype = tf.float32)
        x_tf_data = tf.concat([x_tf_data, [x_next_tf]], 0)
        
        y_next_tf = tf.convert_to_tensor(np.array(y_val),dtype = tf.float32)
        y_tf_data = tf.concat([y_tf_data, [y_next_tf]], 0)
    return (tf.reshape(x_tf_data,(-1,num_day,14,1)),y_tf_data)


In [ ]:
def getMeanAndStd(X,num_day):
    mean_list = []
    std_list = []
    for i in range(0,len(X)-num_day): 
        x_open = X.iloc[i:i+num_day,0]
        mean_list.append(x_open.mean(axis=0))
        std_list.append(x_open.std(axis=0))
    mean_df = pd.DataFrame(mean_list, columns = ["mean"])
    std_df = pd.DataFrame(std_list, columns = ["std"])
    return (mean_df,std_df)

In [ ]:
def get_last_n_day_mean(df,n):
    series = df["open"]
    windows = series.rolling(n)
    n_days_averages = windows.mean()
    n_days_list = n_days_averages.tolist()
    final_list = n_days_list[n:]
    new_df = pd.DataFrame(final_list,columns=["mean_"+str(n)],index=df.index[n:])
    return new_df

# Load Model

In [ ]:
loaded_cnn_lstm_model = keras.models.load_model(model_folder+"cnn_lstm_best/")

# Backtest Result of CNN_LSTM (Price)

In [ ]:
import os

file_list = list(filter(lambda name: "CNN_LSTM_REG_" in name ,os.listdir(record_folder)))
if len(file_list) == 0:
  print("No such files...")
else:
  for f in file_list:
    print(file_list)
    os.remove(record_folder+f)
  print("All old files deleted...")

In [ ]:
#### CNN_LSTM_REGRESSION
backtest_summary = pd.DataFrame(columns=backtest_col)

for sector, stocks in stock_list.items():
    for stock in stocks:
        print("Runninng stock: "+ stock)
        stock_data_1 = web.DataReader(stock, "stooq",start=start_date, end=end_date)
        stock_data_1.columns = ["open","high","low","close","volume"]
        x_1 = StockDataFrame(stock_data_1)
        data_1 = x_1[['open','high','low','close','volume',
                      'boll', 'boll_ub', 'boll_lb',
                      'macd', 'macdh', 'macds',
                      'rsi_11', 'rsi_14', 'rsi_21']]
        data_1.index = [int(str(ind)[0:4]+str(ind)[5:7]+str(ind)[8:10]) for ind in data_1.index]
        data_1 = data_1.sort_index()
        print(stock + " Data loaded...")
        for k,v in date_range_list.items():
            
            print("Getting Data... - " + stock + " from " + str(k))
            if (data_1.index[0]>v):
                print("No data for this testing period... - " + stock + " from " + str(k))
                continue
            test_X_1 = custom_split(data_1,start = k,end = v)
            test_y_1 = produce_result_target_price(test_X_1,num_day_to_predict)
            if len(test_X_1) <= num_day_to_predict:
                print("No enoguh data for testing... - " + stock + " from " + str(k))
                continue
                
            print("Transforming Data... - " + stock + " from " + str(k))
            tf_test_X_1,tf_test_y_1 = transform_data_to_tensor(test_X_1,test_y_1,num_day_to_predict)
            
            print("Predicting... - " + stock + " from " + str(k))
            predictions_1 = loaded_cnn_lstm_model.predict(tf_test_X_1)
            
            print("Converting... - " + stock + " from " + str(k))
            test_mean_1, test_std_1 = getMeanAndStd(test_X_1, num_day_to_predict)
            final_test_y_1 = test_y_1.iloc[num_day_to_predict: , :]
            backtestdata_1 = final_test_y_1.rename(columns={"result_price":"Open"})
            
            final_pred_1 = np.array(predictions_1*np.array(test_std_1) + np.array(test_mean_1))
            compare_to_n_day_mean = 5
            df = get_last_n_day_mean(test_X_1,compare_to_n_day_mean)[(num_day_to_predict-compare_to_n_day_mean):]
            df["pred"] = final_pred_1
            
            df["Action"] = df.apply(lambda row: "Buy" if row["pred"] > row["mean_"+str(compare_to_n_day_mean)] else "Sell", axis=1)
            final_pred_df_1 = df.drop(columns=["mean_"+str(compare_to_n_day_mean),"pred"])
            final_pred_df_1 = final_pred_df_1.sort_index()
            
            print("Backtesting... - " + stock + " from " + str(k))
            back1 = backtest(backtestdata_1,final_pred_df_1,10000,0.0005,period_year)
            back1.clear_trade_record(3)
            
            print("Start Iteration... - " + stock + " from " + str(k))
            back1.start_test()
            
            print("======= from "+ str(k) + " to " + str(v)+" =========")
            back1.print_performance()
            back1.export_trade_record("CNN_LSTM_REG_"+sector+"-"+stock,str(k)+"-to-"+str(v)+"("+str(round(back1.get_performance()*100,2))+"%)")
            backtest_summary = backtest_summary.append({"Sector":sector,
                                                        "Stock":stock,
                                                        "Range_Start":str(k),
                                                        "Range_End":str(v),
                                                        do_nth_name+"_Profit":round(back1.get_do_nothing(),2),
                                                        model_name+"_Profit":round(back1.get_profit(),2),
                                                        do_nth_name+"_CAGR(in %)":round(back1.get_do_nothing_CAGR()*100,2),
                                                        model_name+"_CAGR(in %)":round(back1.get_model_CAGR()*100,2),
                                                        "CAGR_Performance(in %)":round(back1.get_diff_in_CAGR()*100,2),
                                                        do_nth_name+"_MDD(in %)":round(back1.get_do_nothing_MDD()*100,2),
                                                        model_name+"_MDD(in %)":round(back1.get_model_MDD()*100,2),
                                                        "MDD_Performance(in %)":round(back1.get_diff_in_MDD()*100,2)}, ignore_index=True)
            print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

print("Backtesting Completed...")
backtest_summary.to_csv(backtest_summary_folder+"backtest_summary_CNN_LSTM_REG.csv")
print("Backtesting Summary Exported...")
            


Runninng stock: COIN
COIN Data loaded...
Getting Data... - COIN from 20120101
No data for this testing period... - COIN from 20120101
Getting Data... - COIN from 20130101
No data for this testing period... - COIN from 20130101
Getting Data... - COIN from 20140101
No data for this testing period... - COIN from 20140101
Getting Data... - COIN from 20150101
No data for this testing period... - COIN from 20150101
Getting Data... - COIN from 20160101
No data for this testing period... - COIN from 20160101
Getting Data... - COIN from 20170101
No data for this testing period... - COIN from 20170101
Getting Data... - COIN from 20180101
No data for this testing period... - COIN from 20180101
Getting Data... - COIN from 20190101
No data for this testing period... - COIN from 20190101
Getting Data... - COIN from 20200101
No data for this testing period... - COIN from 20200101
Getting Data... - COIN from 20210101
Transforming Data... - COIN from 20210101
Predicting... - COIN from 20210101
Converti

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Streaming output truncated to the last 5000 lines.
Bought at 20180419 with price = 2175.49 	Pos: 5
Sold at 20180423 with price = 2133.1 	Pos: 5
Bought at 20180425 with price = 2061.12 	Pos: 5
Sold at 20180503 with price = 2149.0 	Pos: 5
Bought at 20180508 with price = 2195.02 	Pos: 5
Sold at 20180509 with price = 2185.0 	Pos: 5
Bought at 20180510 with price = 2050.0 	Pos: 5
Sold at 20180522 with price = 2076.03 	Pos: 5
Bought at 20180523 with price = 2056.05 	Pos: 5
Sold at 20180530 with price = 2086.41 	Pos: 5
Bought at 20180604 with price = 2134.0 	Pos: 5
Sold at 20180608 with price = 2127.71 	Pos: 5
Bought at 20180612 with price = 2126.09 	Pos: 5
Sold at 20180620 with price = 2149.0 	Pos: 5
Bought at 20180621 with price = 2145.54 	Pos: 5
Sold at 20180622 with price = 2130.04 	Pos: 5
Bought at 20180625 with price = 2091.26 	Pos: 5
Sold at 20180724 with price = 2009.0 	Pos: 5
Bought at 20180725 with price = 2029.48 	Pos: 5
Sold at 20180726 with price = 2067.92 	Pos: 5
Bought at 201807

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Streaming output truncated to the last 5000 lines.
Bought at 20180427 with price = 14.75 	Pos: 583
Sold at 20180525 with price = 10.66 	Pos: 583
Bought at 20180529 with price = 10.68 	Pos: 582
Sold at 20180531 with price = 11.17 	Pos: 582
Bought at 20180601 with price = 11.71 	Pos: 554
Sold at 20180706 with price = 13.17 	Pos: 554
Bought at 20180709 with price = 13.66 	Pos: 534
Sold at 20180712 with price = 13.43 	Pos: 534
Bought at 20180713 with price = 13.6 	Pos: 527
Sold at 20180718 with price = 13.4 	Pos: 527
Bought at 20180719 with price = 13.3 	Pos: 530
Sold at 20180724 with price = 13.45 	Pos: 530
Bought at 20180725 with price = 13.32 	Pos: 535
Sold at 20180816 with price = 12.3 	Pos: 535
Bought at 20180820 with price = 11.8 	Pos: 557
Sold at 20180925 with price = 9.19 	Pos: 557
Bought at 20180926 with price = 9.09 	Pos: 563
Sold at 20181018 with price = 6.96 	Pos: 563
Bought at 20181019 with price = 6.89 	Pos: 568
Sold at 20181106 with price = 6.86 	Pos: 568
Bought at 20181107 

...

Start Iteration... - EQR from 20210101
Start Backtesting...
Bought at 20210111 with price = 56.026 	Pos: 178
Sold at 20210114 with price = 57.74 	Pos: 178
Bought at 20210115 with price = 58.251 	Pos: 176
Sold at 20210202 with price = 59.995 	Pos: 176
Bought at 20210203 with price = 59.32 	Pos: 178
Sold at 20210205 with price = 62.134 	Pos: 178
Bought at 20210208 with price = 62.587 	Pos: 176
Sold at 20210218 with price = 64.118 	Pos: 176
Bought at 20210219 with price = 64.177 	Pos: 176
Sold at 20210226 with price = 64.293 	Pos: 176
Bought at 20210301 with price = 64.148 	Pos: 176
Sold at 20210318 with price = 70.901 	Pos: 176
Bought at 20210322 with price = 68.55 	Pos: 182
Sold at 20210401 with price = 69.965 	Pos: 182
Bought at 20210406 with price = 69.868 	Pos: 182
Sold at 20210414 with price = 70.071 	Pos: 182
Bought at 20210415 with price = 69.673 	Pos: 183
Sold at 20210514 with price = 71.85 	Pos: 183
Bought at 20210517 with price = 71.946 	Pos: 183
Sold at 20210520 with price = 7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Start Iteration... - AVB from 20140101
Start Backtesting...
Bought at 20140109 with price = 91.724 	Pos: 108
Sold at 20140110 with price = 93.211 	Pos: 108
Bought at 20140113 with price = 93.201 	Pos: 108
Sold at 20140122 with price = 94.105 	Pos: 108
Bought at 20140123 with price = 93.098 	Pos: 109
Sold at 20140214 with price = 98.6 	Pos: 109
Bought at 20140220 with price = 98.917 	Pos: 109
Sold at 20140221 with price = 98.897 	Pos: 109
Bought at 20140225 with price = 98.967 	Pos: 109
Sold at 20140306 with price = 98.967 	Pos: 109
Bought at 20140310 with price = 98.442 	Pos: 109
Sold at 20140321 with price = 99.81 	Pos: 109
Bought at 20140324 with price = 100.04 	Pos: 109
Sold at 20140403 with price = 106.71 	Pos: 109
Bought at 20140404 with price = 102.95 	Pos: 113
Sold at 20140409 with price = 103.65 	Pos: 113
Bought at 20140410 with price = 103.44 	Pos: 113
Sold at 20140421 with price = 103.5 	Pos: 113
Bought at 20140424 with price = 102.64 	Pos: 114
Sold at 20140514 with price = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Start Iteration... - VZ from 20210101
Start Backtesting...
Bought at 20210111 with price = 54.51 	Pos: 183
Sold at 20210210 with price = 52.122 	Pos: 183
Bought at 20210212 with price = 51.369 	Pos: 185
Sold at 20210217 with price = 52.858 	Pos: 185
Bought at 20210218 with price = 53.747 	Pos: 182
Sold at 20210223 with price = 53.624 	Pos: 182
Bought at 20210224 with price = 53.879 	Pos: 181
Sold at 20210312 with price = 52.603 	Pos: 181
Bought at 20210315 with price = 52.681 	Pos: 181
Sold at 20210319 with price = 53.263 	Pos: 181
Bought at 20210322 with price = 52.736 	Pos: 182
Sold at 20210401 with price = 54.765 	Pos: 182
Bought at 20210406 with price = 55.512 	Pos: 180
Sold at 20210409 with price = 54.953 	Pos: 180
Bought at 20210412 with price = 55.01 	Pos: 179
Sold at 20210514 with price = 56.297 	Pos: 179
Bought at 20210517 with price = 56.184 	Pos: 179
Sold at 20210610 with price = 54.696 	Pos: 179
Bought at 20210611 with price = 54.848 	Pos: 179
Sold at 20210616 with price = 